In [29]:
import tushare as ts
import pandas as pd
import pickle

In [82]:
#Used to import HS300 index component ticker list from excel file, which can be downloaded from http://www.csindex.com.cn/.

index_sheet = pd.read_excel('D:/Downloads/000300closeweight.xls', dtype = str)
ticker_list = index_sheet.values[:, 4]
exchange_list = index_sheet.values[:, 7]
for i in range(len(ticker_list)):
    if(exchange_list[i] == 'SHH'):
        ticker_list[i] = ticker_list[i] + '.SH'
    elif(exchange_list[i] == 'SHZ'):
        ticker_list[i] = ticker_list[i] + '.SZ'
my_file = open('./hs300_components', 'wb')
pickle.dump(ticker_list, my_file)
my_file.close()

In [83]:
my_file = open('./hs300_components', 'rb')
ticker_list = pickle.load(my_file)
my_file.close()

In [2]:
ts.set_token('e4cba0aba52adb6aef525f4363b9e19e01dc6e02a3b46f97c882b14e')

In [4]:
pro = ts.pro_api()

In [85]:
def tushare_qfq_to_zipline_csv(ticker, start_date, end_date, directory):
    df = ts.pro_bar(ts_code = ticker, adj = 'qfq', start_date = start_date, end_date = end_date)
    df['date'] = pd.to_datetime(df['trade_date'])
    df['volume'] = df['amount']
    df = df.set_index('date')
    df = df.drop(labels = ['ts_code', 'trade_date', 'pre_close', 'change', 'pct_chg', 'vol', 'amount'], axis = 1)
    df = df.sort_index()
    df.to_csv(directory)
    return 0

In [86]:
start_date = '20160630'
end_date = '20190630'

for ticker in ticker_list:
    name = ticker.split('.')[0]
    directory = './china_a_stocks/daily/' + name + '.csv'
    tushare_qfq_to_zipline_csv(ticker, start_date, end_date, directory)